# Projet 6 Machine Learning (suite)

## Transfer learning

Le transfer learning consiste à utiliser un réseau de neurones préalablement entraîné sur des millions d’images de toute sorte. Le principal intérêt de ce procédé est le pouvoir de généralisation car les features apprises peuvent s’avérer utiles pour bon nombre de classifications.

Les réseaux VGG16 et effnet seront testés.

In [16]:
# Import du réseau VGG16

pretrained_model = keras.applications.vgg16.VGG16(weights="imagenet", include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [17]:
# Gel de tous les layers du réseau

pretrained_model.trainable = False

In [18]:
# On ajoute au modèle pré entraîné des layers dont un layer de classification pour l'adapter à notre problème

def build_vgg16_model(hp):
    inputs = keras.Input(shape=(180, 180,3))
    x = keras.applications.vgg16.preprocess_input(inputs)
    x = pretrained_model(x)
    x = layers.Flatten()(x)
    unit = hp.Int(name="unit", min_value=64, max_value=512, step=64)
    x = layers.Dense(unit)(x)
    rate = hp.Choice('rate', values=[0.5, 0.6, 0.7])
    x = layers.Dropout(rate)(x)
    outputs = layers.Dense(6, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)    
    hp_learning_rate = hp.Choice('learning_rate', values=[0.0001, 0.00001, 0.00005], ordered=False)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [19]:
import kerastuner as kt
tuner = kt.BayesianOptimization(
build_vgg16_model,
objective="val_accuracy",
max_trials=20,executions_per_trial=2,
directory="dogs_classifications",
overwrite=True)

In [20]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
unit (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 64, 'sampling': None}
rate (Choice)
{'default': 0.5, 'conditions': [], 'values': [0.5, 0.6, 0.7], 'ordered': True}
learning_rate (Choice)
{'default': 0.0001, 'conditions': [], 'values': [0.0001, 1e-05, 5e-05], 'ordered': False}


In [21]:
callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)]


In [22]:
tuner.search(train_dataset, batch_size=32, epochs=30, validation_data = val_dataset,
             callbacks=callbacks, verbose = 2)

Trial 20 Complete [00h 00m 47s]
val_accuracy: 0.9285714328289032

Best val_accuracy So Far: 0.9489796161651611
Total elapsed time: 00h 21m 09s


On voit que notre modèle a une efficacité de 0.95. Comparé à celle de nos modèles classiques qui frolent les 0.40, cela représente un gain considérable.

In [32]:
best_hps= tuner.get_best_hyperparameters()[0]

In [34]:
best_hps.values

{'unit': 512, 'rate': 0.7, 'learning_rate': 5e-05}

In [38]:
def get_best_epoch(hp):
    model = build_vgg16_model(hp)
    callbacks=[
    keras.callbacks.EarlyStopping(
    monitor="val_loss", mode="min", patience=10)]
    
    history = model.fit(train_dataset, batch_size=32, epochs=30, validation_data = val_dataset,
             callbacks=callbacks, verbose = 2)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [39]:
best_model = get_best_epoch(best_hps)

Epoch 1/30


/opt/conda/lib/python3.7/site-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


26/26 - 2s - loss: 46.7017 - accuracy: 0.3772 - val_loss: 9.0515 - val_accuracy: 0.7653
Epoch 2/30
26/26 - 2s - loss: 13.5249 - accuracy: 0.7370 - val_loss: 6.7480 - val_accuracy: 0.8061
Epoch 3/30
26/26 - 2s - loss: 6.5816 - accuracy: 0.8400 - val_loss: 4.3183 - val_accuracy: 0.8878
Epoch 4/30
26/26 - 2s - loss: 5.7403 - accuracy: 0.8561 - val_loss: 4.0192 - val_accuracy: 0.8673
Epoch 5/30
26/26 - 2s - loss: 4.2490 - accuracy: 0.8846 - val_loss: 3.5973 - val_accuracy: 0.8776
Epoch 6/30
26/26 - 2s - loss: 3.3001 - accuracy: 0.9206 - val_loss: 3.9930 - val_accuracy: 0.8980
Epoch 7/30
26/26 - 2s - loss: 2.7202 - accuracy: 0.9206 - val_loss: 3.7741 - val_accuracy: 0.8878
Epoch 8/30
26/26 - 2s - loss: 2.3381 - accuracy: 0.9355 - val_loss: 3.7312 - val_accuracy: 0.8878
Epoch 9/30
26/26 - 2s - loss: 1.6569 - accuracy: 0.9541 - val_loss: 4.2846 - val_accuracy: 0.8980
Epoch 10/30
26/26 - 2s - loss: 1.3103 - accuracy: 0.9541 - val_loss: 3.6373 - val_accuracy: 0.9082
Epoch 11/30
26/26 - 2s - los

In [40]:
# Import du réseau VGG16

effnet_model = tf.keras.applications.EfficientNetB7(include_top=False,weights="imagenet")

258088960/258076736 [==============================] - 2s 0us/step


In [41]:
# Gel de tous les layers

effnet_model.trainable = False

In [45]:
# Même chose que pour le modèle VGG16

def build_effnet_model(hp):
    inputs = keras.Input(shape=(180, 180,3))
    x = effnet_model(inputs)
    x = layers.Flatten()(x)
    unit = hp.Int(name="unit", min_value=64, max_value=512, step=64)
    x = layers.Dense(unit)(x)
    rate = hp.Choice('rate', values=[0.5, 0.6, 0.7])
    x = layers.Dropout(rate)(x)
    outputs = layers.Dense(6, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)    
    hp_learning_rate = hp.Choice('learning_rate', values=[0.0001, 0.00001, 0.00005], ordered=False)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

In [47]:
import kerastuner as kt
tuner = kt.BayesianOptimization(
build_effnet_model,
objective="val_accuracy",
max_trials=20,executions_per_trial=2,
directory="dogs_classifications",
overwrite=True)

In [48]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
unit (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 64, 'sampling': None}
rate (Choice)
{'default': 0.5, 'conditions': [], 'values': [0.5, 0.6, 0.7], 'ordered': True}
learning_rate (Choice)
{'default': 0.0001, 'conditions': [], 'values': [0.0001, 1e-05, 5e-05], 'ordered': False}


In [49]:
callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)]


In [50]:
tuner.search(train_dataset, batch_size=32, epochs=30, validation_data = val_dataset,
             callbacks=callbacks, verbose = 2)

Trial 20 Complete [00h 02m 06s]
val_accuracy: 0.954081654548645

Best val_accuracy So Far: 0.9744898080825806
Total elapsed time: 00h 52m 17s


Le modèle avec les hyperparamètres optimisés a une accuracy max de 0.97, ce qui est encore meilleur reposant sur VGG16. Il sera choisi comme modèle pour la généralisation à toutes les races.

In [52]:
best_hps= tuner.get_best_hyperparameters()[0]

In [53]:
best_hps.values

{'unit': 384, 'rate': 0.5, 'learning_rate': 0.0001}

In [55]:
model = build_effnet_model(best_hps)

In [56]:
def get_best_epoch(hp):
    model = build_effnet_model(hp)
    callbacks=[
    keras.callbacks.EarlyStopping(
    monitor="val_loss", mode="min", patience=10)]
    
    history = model.fit(train_dataset, batch_size=32, epochs=30, validation_data = val_dataset,
             callbacks=callbacks, verbose = 2)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [57]:
best_model = get_best_epoch(best_hps)

Epoch 1/30
26/26 - 28s - loss: 0.8696 - accuracy: 0.8325 - val_loss: 0.2940 - val_accuracy: 0.9592
Epoch 2/30
26/26 - 5s - loss: 0.2227 - accuracy: 0.9615 - val_loss: 0.3397 - val_accuracy: 0.9592
Epoch 3/30
26/26 - 5s - loss: 0.1801 - accuracy: 0.9628 - val_loss: 0.4631 - val_accuracy: 0.9490
Epoch 4/30
26/26 - 5s - loss: 0.0829 - accuracy: 0.9814 - val_loss: 0.4754 - val_accuracy: 0.9490
Epoch 5/30
26/26 - 5s - loss: 0.0575 - accuracy: 0.9901 - val_loss: 0.4067 - val_accuracy: 0.9592
Epoch 6/30
26/26 - 5s - loss: 0.0822 - accuracy: 0.9851 - val_loss: 0.5246 - val_accuracy: 0.9592
Epoch 7/30
26/26 - 5s - loss: 0.0927 - accuracy: 0.9839 - val_loss: 0.6692 - val_accuracy: 0.9592
Epoch 8/30
26/26 - 5s - loss: 0.0745 - accuracy: 0.9888 - val_loss: 0.5096 - val_accuracy: 0.9490
Epoch 9/30
26/26 - 5s - loss: 0.1398 - accuracy: 0.9814 - val_loss: 0.6467 - val_accuracy: 0.9592
Epoch 10/30
26/26 - 5s - loss: 0.0888 - accuracy: 0.9864 - val_loss: 0.7415 - val_accuracy: 0.9490
Epoch 11/30
26/26 

# Modèle final

On entraîne le modèle retenu pour la généralisation sur toutes les races de chiens. Il est attendu que son accuracy va baisser assez nettement.

In [74]:
def build_final_model(hp):
    inputs = keras.Input(shape=(180, 180,3))
    x = effnet_model(inputs)
    x = layers.Flatten()(x)
    unit = hp.Int(name="unit", min_value=64, max_value=512, step=64)
    x = layers.Dense(unit)(x)
    rate = hp.Choice('rate', values=[0.5, 0.6, 0.7])
    x = layers.Dropout(rate)(x)
    outputs = layers.Dense(120, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)    
    hp_learning_rate = hp.Choice('learning_rate', values=[0.0001, 0.00001, 0.00005], ordered=False)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

In [75]:
final_model = build_final_model(best_hps)

In [59]:
%mkdir dataset

In [68]:
cp -R '/kaggle/input/stanford-dogs-dataset/images/Images/' '/root/dataset'

In [71]:
splitfolders.ratio('/root/dataset/', output="/root/dataset/output", seed=1337, ratio=(.8, .1,.1))


Copying files: 20580 files [00:04, 5111.61 files/s]


In [72]:
train_dataset = tf.keras.utils.image_dataset_from_directory (
            '/root/dataset/output/train',
            image_size=(180, 180),
            batch_size=32)
val_dataset = tf.keras.utils.image_dataset_from_directory(
            '/root/dataset/output/val',
            image_size=(180, 180),
            batch_size=32)
test_dataset = tf.keras.utils.image_dataset_from_directory(
            '/root/dataset/output/test',
            image_size=(180, 180),
            batch_size=32)

Found 16418 files belonging to 120 classes.
Found 2009 files belonging to 120 classes.
Found 2153 files belonging to 120 classes.


In [78]:
callbacks = [
keras.callbacks.ModelCheckpoint(
                                filepath="final_model",
                                save_best_only=True,
                                monitor="val_loss")]

In [83]:
history = final_model.fit(train_dataset, epochs=20,validation_data=val_dataset, batch_size=32, callbacks=callbacks)

Epoch 1/20


/opt/conda/lib/python3.7/site-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


513/514 [============================>.] - ETA: 0s - loss: 2.5976 - accuracy: 0.4097

Cleanup called...


514/514 [==============================] - 115s 191ms/step - loss: 2.5974 - accuracy: 0.4097 - val_loss: 1.1621 - val_accuracy: 0.6904


...

514/514 [==============================] - 94s 183ms/step - loss: 0.1589 - accuracy: 0.9555 - val_loss: 1.0857 - val_accuracy: 0.7252
Epoch 19/20
513/514 [============================>.] - ETA: 0s - loss: 0.1472 - accuracy: 0.9590

Cleanup called...


514/514 [==============================] - 94s 183ms/step - loss: 0.1472 - accuracy: 0.9590 - val_loss: 1.0984 - val_accuracy: 0.7347
Epoch 20/20
513/514 [============================>.] - ETA: 0s - loss: 0.1366 - accuracy: 0.9623

Cleanup called...


514/514 [==============================] - 94s 182ms/step - loss: 0.1366 - accuracy: 0.9623 - val_loss: 1.0685 - val_accuracy: 0.7317


On voit qu'il a en moyenne une accuracy d'un peu plus de 0.70, ce qui en fait un modèle plus que fiable de détection de race de chien 